Instalación de bibliotecas

In [10]:
!pip install boxoffice_api
!pip install rottentomatoes-python

Defaulting to user installation because normal site-packages is not writeable


Importación de bibliotecas

In [92]:
from boxoffice_api import BoxOffice
import rottentomatoes as rt
import datetime
from sqlalchemy import create_engine
import pandas as pd
import psycopg2

Obtención de datos de la API

In [106]:
#Obtengo el password para la API
with open("C:/Users/PC2/Desktop/rotten_api.txt",'r') as f:
    pwd= f.read()

In [53]:
#Inicialización de la API
box_office = BoxOffice(api_key=pwd,outputformat="DF")

In [54]:
#Seteo de datos de fecha
hoy = datetime.date.today()#.strftime("%Y-%m-%d")

In [69]:
#Obtengo los datos de la API
df = box_office.get_monthly(year=hoy.year,month=hoy.month)
cols = df.columns[:-2]
df = df[cols]
df = df.rename(columns={'Total Gross':'Total_Gross','Release Date':'Release_Date'})

Agrego datos extra al DF

In [70]:

df['Year'] = hoy.year
df['Month'] = hoy.month
df['RottenTomatoes_Score'] = [rt.tomatometer(x) for x in df['Release']]
df['Audience_Score'] = [rt.audience_score(x) for x in df['Release']]
df['Genre'] = [rt.genres(x) for x in df['Release']]

Conexión a DB

In [90]:
#Obtengo el password para Redshift
with open("C:/Users/PC2/Desktop/redshift_info.txt",'r') as f:
    pwd= f.read()

In [104]:
#Conectando a la DB
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname='data-engineer-database',
        user='pdiazmedin_coderhouse',
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


Creación de la tabla en Redshift

In [105]:
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS pdiazmedin_coderhouse.peliculas
        ( 
		Rank INTEGER,
		Release VARCHAR(255),
		Gross VARCHAR(255),
		Theaters VARCHAR(255),
		Total_Gross VARCHAR(255),
		Release_Date VARCHAR(255),
		Distributor VARCHAR(255),
		Year INTEGER,
		Month INTEGER,
		RottenTomatoes_Score INTEGER,
		Audience_Score INTEGER,
		Genre VARCHAR(255)
        )
    """)
    conn.commit()